In [9]:
!pip install -e ./


Obtaining file:///home/diego.fan/Documents/dev_work/ibge_gavb
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ibge-gavb
    Found existing installation: ibge-gavb 1.0.0
    Uninstalling ibge-gavb-1.0.0:
      Successfully uninstalled ibge-gavb-1.0.0
  Running setup.py develop for ibge-gavb


In [10]:
from ibge_gavb.localidades import *

In [11]:
data = Estados()

SSLError: HTTPSConnectionPool(host='servicodados.ibge.gov.br', port=443): Max retries exceeded with url: /api/v1/localidades/estados (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:992)')))

In [4]:
import requests
import urllib3
import ssl
import json


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [1]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes'
request = get_legacy_session().get(url)
json_ibge = json.loads(request.content.decode('utf-8'))

NameError: name 'get_legacy_session' is not defined

In [5]:
import requests
import urllib3
import json

URL = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes'

class TestAdapter(requests.adapters.HTTPAdapter):
    def __init__(self, response):
        self._response = response
        super(TestAdapter, self).__init__()

    def send(self, request, *args, **kwargs):
        return self.build_response(request, self._response)

resp = urllib3\
    .HTTPResponse(body=json.dumps({'results': results}), status=200,
                  headers={'content-type': 'application/json'})
adapter = TestAdapter(resp)
session = requests.Session()
session.mount(URL, adapter)

response = session.post(URL)
response.json()

NameError: name 'results' is not defined